# Data Extraction from REST API

## Libraries installation

In [ ]:
#install the required libraries
pip install requests
pip install pandas
pip install boto3
pip install awswrangler
pip install numpy
pip install pandas sqlalchemy pyodbc
pip install psycopg2-binary

## Libraries importation

In [ ]:
#importing the required libraries
import pandas as pd
import requests
import boto3
import awswrangler as wr
import numpy
from sqlalchemy import create_engine
import psycopg2

## Data extraction using python request

In [ ]:
# extraction using python request method
response = requests.get("https://restcountries.com/v3.1/all")

In [ ]:
#checking the status code of the responses receive by the server
response.status_code

In [ ]:
response = response.json()

In [ ]:
#returning the first list to investigate the items
response[0]

In [ ]:
#extracting the relevant columns 
extracted_datas = []
for item in response:
    name_info = item.get('name',{})
    country_name = name_info.get('common','')
    common_native_name = name_info.get('nativeName',{}).get('eng',{}).get('official','')
    official_country_name = name_info.get('official','')
    independence = item.get('independent','')
    united_Nation_members = item.get('unMember','')
    startOfWeek = item.get('startOfWeek','')
    capital = item.get('capital',[''])[0]
    region = item.get('region','')
    sub_region = item.get('subregion','')
    areas = item.get('area','')
    population = item.get('population','')
    continents = item.get('continents',[''])[0]
    languages = ', '.join(item.get('languages',{}).values())
    idd_info = item.get('idd',{})
    idd_root = idd_info.get('root','')
    idd_suffix = ''.join(idd_info.get('suffixes',[]))
    country_code = idd_root + idd_suffix
    currency_info = item.get('currencies',{})
    currency_code = next(iter(currency_info),'')
    currency_name = currency_info.get(currency_code,{}).get('name','')
    currency_symbol = currency_info.get(currency_code,{}).get('symbol','')
    
# appending the extracted data into the empty list created
    extracted_datas.append({
        'country_name': country_name,
        'common_native_name': common_native_name,
        'official_country_name': official_country_name,
        'independence': independence,
        'united_Nation_members': united_Nation_members,
        'startOfWeek': startOfWeek,
        'currency_name': currency_name,
        'currency_symbol':currency_symbol,
        'capital':capital,
        'region':region,
        'sub_region': sub_region,
        'languages':languages,
        'areas': areas,
        'population': population,
        'continents': continents,
        'currency_code':currency_code,
        'country_code': country_code
        
    })
# Converted to DataFrame for easier manipulation
df=pd.DataFrame(extracted_datas)
df

# Writing the dataframe to a postgres db hosted in AWS cloud  

## Retrieving the database password from AWS SSM parameter

In [ ]:
# Initialize a session using Boto3
session = boto3.Session(region_name='eu-central-1')

In [ ]:
# Initialize the SSM client
ssm_client = session.client('ssm')

In [ ]:
# retrieving the parameter value
parameter_name = 'DEC_hackathon_db_pwd'
parameter = ssm_client.get_parameter(Name=parameter_name, WithDecryption=True)
db_password = parameter['Parameter']['Value']

In [ ]:
# creating a connection engine

db_username = 'bimmy'
db_name = 'dec_hackathon_db'
db_host = 'dec-hackathon.c7a2yi4u263m.eu-central-1.rds.amazonaws.com' 
db_port = 5432

engine = create_engine(f'postgresql+psycopg2://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')


In [ ]:
# Writing the DataFrame to PostgreSQL table
df.to_sql('dec_hackathon_tb', engine, if_exists='replace', index=False)